In [1]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from claspy.segmentation import BinaryClaSPSegmentation
from claspy.data_loader import load_tssb_dataset, load_has_dataset
import ruptures as rpt 
from utilsCPD import *
import time

In [2]:
df_TS_CL = load_has_dataset()

In [14]:
METRICS = {}
METRICS_25 = {}
METRICS_50 = {}
METRICS_150 = {}
METRICS_200 = {}
IDs = [10,14,7,182,225,19,185,33,36,87,88,210,11,20,23,243,247,91,95,96,100,141,91,95,245]


for id in IDs:
    print(id)
    METRICS[id]  = {}
    METRICS_25[id]  = {}
    METRICS_50[id]  = {}
    METRICS_150[id]  = {}
    METRICS_200[id]  = {}
    dataset, window_size, true_cps,labels, time_series = df_TS_CL.iloc[id,:]
    algo_c = rpt.KernelCPD(kernel="rbf", min_size=2).fit(
        time_series
    ) 
    penalty_value = 10 # beta
    cps_KCPA_PELT = algo_c.predict(pen=penalty_value)
    cps = np.array(cps_KCPA_PELT)[:-1]
    f1,AUC_run,fp = f_measure({'0':true_cps},cps,100)
    METRICS[id]['F1'] = f1
    METRICS[id]['AUC'] = AUC_run
    METRICS[id]['FP'] = fp
    METRICS[id]['Covering'] = covering({'0':true_cps},cps,time_series.shape[0])
    METRICS[id]['DD'] = detection_delay(true_cps,cps)[1]

    f1,AUC_run,fp = f_measure({'0':true_cps},cps,25)
    METRICS_25[id]['F1'] = f1
    METRICS_25[id]['AUC'] = AUC_run
    METRICS_25[id]['FP'] = fp
    METRICS_25[id]['Covering'] = covering({'0':true_cps},cps,time_series.shape[0])
    METRICS_25[id]['DD'] = detection_delay(true_cps,cps)[1]

    f1,AUC_run,fp = f_measure({'0':true_cps},cps,50)
    METRICS_50[id]['F1'] = f1
    METRICS_50[id]['AUC'] = AUC_run
    METRICS_50[id]['FP'] = fp
    METRICS_50[id]['Covering'] = covering({'0':true_cps},cps,time_series.shape[0])
    METRICS_50[id]['DD'] = detection_delay(true_cps,cps)[1]


    f1,AUC_run,fp = f_measure({'0':true_cps},cps,150)
    METRICS_150[id]['F1'] = f1
    METRICS_150[id]['AUC'] = AUC_run
    METRICS_150[id]['FP'] = fp
    METRICS_150[id]['Covering'] = covering({'0':true_cps},cps,time_series.shape[0])
    METRICS_150[id]['DD'] = detection_delay(true_cps,cps)[1]

    f1,AUC_run,fp = f_measure({'0':true_cps},cps,200)
    METRICS_200[id]['F1'] = f1
    METRICS_200[id]['AUC'] = AUC_run
    METRICS_200[id]['FP'] = fp
    METRICS_200[id]['Covering'] = covering({'0':true_cps},cps,time_series.shape[0])
    METRICS_200[id]['DD'] = detection_delay(true_cps,cps)[1]







F1,Covering,AUC,DD,FP = [],[],[],[],[]

for k,metrics in METRICS.items():
    if len(metrics)>1:
        F1.append(metrics['F1'])
        Covering.append(metrics['Covering'])
        AUC.append(metrics['AUC'])
        DD.append(metrics['DD'])
        FP.append(metrics['FP'])



def AUC_analysis(METRICS_SWD):
    f1,cov,AUC,DD,FP = [],[],[],[],[]
    for k,metrics in METRICS_SWD.items():
        if len(metrics)>1:
            f1.append(metrics['F1'])
            cov.append(metrics['Covering'])
            AUC.append(metrics['AUC'])
            DD.append(metrics['DD'])
            FP.append(metrics['FP'])

    return np.mean(AUC),np.mean(FP)

MEAN ,FPs= [], []

for res in [METRICS_25,METRICS_50,METRICS,METRICS_150,METRICS_200]:
    m,fp =AUC_analysis(res)
    MEAN.append(m)
    FPs.append(fp)

MEAN,FPs

10
14
7
182
225
19
185
33
36
87
88
210
11
20
23
243
247
91
95
96
100
141
91
95
245


([0.6093832204863007,
  0.6370112084403755,
  0.6571305605290371,
  0.6571305605290371,
  0.6571305605290371],
 [15.782608695652174,
  15.043478260869565,
  13.956521739130435,
  12.565217391304348,
  11.652173913043478])

In [12]:
print('AUC',np.mean(AUC),np.std(AUC))
print('COV',np.mean(Covering),np.std(Covering))
print('DD',np.mean(DD),np.std(DD),np.min(DD),np.max(DD))
print('FP',np.mean(FP),np.std(FP),np.min(FP),np.max(FP))

AUC 0.6571305605290371 0.1433144575916974
COV 0.585138565707187 0.3198593082526968
DD 334.4526572522555 428.20860009809405 0.0 1540.7948717948718
FP 13.956521739130435 14.50180876535471 0 47


In [13]:
Occupancy = pd.read_csv('./datasets/Occupancy/Occupancy.csv').iloc[:,1:]
algo_c = rpt.KernelCPD(kernel="rbf", min_size=2).fit(
        Occupancy.to_numpy()
    ) 
penalty_value = 50 # beta
cps_KCPA_PELT = algo_c.predict(pen=penalty_value)
cps = np.array(cps_KCPA_PELT)[:-1]
print(cps)
GroundTruth = [15,831,1455,2271,2895,3743,4271,5183,5759,6655,7215,8095]
f1,AUC_run,fp = f_measure({'0':GroundTruth},cps,30)
print('COV',covering({'0':GroundTruth},cps,Occupancy.shape[0]))
print('AUC',AUC_run)
print('FP',fp)
print('DD',detection_delay(GroundTruth,cps)[1])



KCPA_AUCS = [f_measure({'0':GroundTruth},cps,tau)[2] for tau in [10,30,50,75,100]]

KCPA_AUCS

[ 153  827  919 1454 1527 2269 2358 2896 2960 3806 4107 6654 6742 7035
 7213 7364 7492 8086]
COV 0.6407256925917354
AUC 0.5182186234817814
FP 11
DD 77.22222222222223


[11, 11, 11, 8, 5]

In [5]:
import zipfile
import json

zip_file_path = './datasets/MNISTSeq.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_contents = zip_ref.namelist()
    json_file_name = zip_contents[0]
    with zip_ref.open(json_file_name) as json_file:
        data = json.load(json_file)

In [11]:

F1,Covering,AUC,DD,FP,RT = [],[],[],[],[],[]
F1_5 = []
F1_10 = []
F1_15 = []
F1_30 = []
for id, k in enumerate(data.keys()):
    ground_truths = data[k]['target']
    ts = data[k]['data']
    n = ground_truths[-1] + 200
    algo_c = rpt.KernelCPD(kernel="rbf", min_size=1).fit(
        np.array(ts)
    ) 
    penalty_value = 1# beta
    start_time = time.time()
    cps_KCPA_PELT = algo_c.predict(pen=penalty_value)
    cps = np.array(cps_KCPA_PELT)[:-1]
    print(cps)
    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    delay = detection_delay(ground_truths,cps)[1]
    end_time = time.time()
    RT.append(end_time-start_time)
    F1.append(f1)
    AUC.append(AUC_score)
    Covering.append(covering_score)
    DD.append(delay)
    FP.append(fp)

    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,5)
    F1_5.append(fp)

    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,10)
    F1_10.append(fp)

    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,15)
    F1_15.append(fp)

    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,30)
    F1_30.append(fp)


print('AUC',np.mean(AUC),np.std(AUC))
print('COV',np.mean(Covering),np.std(Covering))
print('DD',np.mean(DD),np.min(DD),np.max(DD))
print('FP',np.mean(FP),np.min(FP),np.max(FP))
print('RT',np.mean(RT),np.std(RT))


[200 400]
[  4 200 374 381 400]
[200 400]
[  4 200 287 400]
[200 400]
[200 287 400 600]
[200 400 404 600 774 781]
[ 87 200 204 400 600]
[200 287 400 600]
[  4 200 287 400 600]
[200 204 400 600 687 800]
[174 181 200 400 600 800]
[200 400 600 800]
[200 204 400 600 800]
[200 400 600 800]
AUC 0.8955555555555555 0.08489407684399873
COV 0.9463222222222224 0.05303691819250082
DD 21.716666666666672 0.0 70.75
FP 0.6666666666666666 0 2
RT 0.03258961041768392 0.008632984247586958


In [12]:
[np.mean(f) for f in [F1_5,F1_10,F1_15,FP,F1_30]]

[0.8, 0.8, 0.8, 0.6666666666666666, 0.5333333333333333]